# Replication of attack on VoIP end-to-end encrypted messengers

## Models

### Loading and preprocessing

We will now try to explore various models on `Whatsapp` dataset. Bellow we will find loading and preprocessing that we have come up with in the analysis section.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

sns.set()  # make plots nicer

np.random.seed(42)  # set seed for reproducibility

In [2]:
def file_parser_with_prev_next(path):
    file = open(path, 'r')
    lines = file.readlines()
    
    file_name = [path.split('/')[-1]]
    sentence = ""
    file_data = []
    
    has_value = False
    previous = 0
    
    for line in lines:
        line = line.strip()
        
        # if there are only 2 informations on line and second is h#, then ignore
        # strip line, split primarly on ; secondary on ,
        if (line.startswith('#')):
            if (not sentence):
                sentence = line[len('# Sentence: "'): len(line) - 1]
            continue
        
        line = line.split(';')
        
        if (len(line) == 1):
            #lines containing only their packet size and nothing else, they should be added
            #TODO
            line += [""]
            line += [""]
            #continue
        
        if (len(line) == 2):
            #this tries to remove most of the silence at the start of the recording
            #potentionally harmfull as we shouldn't clean test data this way (we will be reading labels)
            #if (line[1] == 'h#'):
            #    continue
            line += [""]
        
        line[1] = tuple(line[1].split(','))
        line[2] = tuple(list(map(lambda a: a.strip('"'), line[2].split(','))))
        
        if (has_value):
            file_data[-1][4] = line[0]
           
        # file_type and sentence contain duplicate informations, but are kept for readability
        line = file_name + [file_name[0][0:9]] + [sentence] + [previous] + [0] + line
        #adding previous as feature
        previous = line[5]
        file_data += [line]
        
        #adding next frame as feature
        has_value = True
        
        

    return pd.DataFrame(file_data, columns=['file', 'speaker', 'sentence', 'previous_packet', 'next_packet','packet_size', 'phonemes', 'words'])

def load_files_with_prev_next(directory):
    filelist = os.listdir(directory)
    #read them into pandas
    df_list = [file_parser_with_prev_next(directory+file) for file in filelist]
    #concatenate them together
    return pd.concat(df_list, ignore_index=True)

In [3]:
def convert_types(data_frame):
    data_frame['packet_size'] = pd.to_numeric(data_frame['packet_size'])
    data_frame['previous_packet'] = pd.to_numeric(data_frame['previous_packet'])
    data_frame['next_packet'] = pd.to_numeric(data_frame['next_packet'])

    data_frame['file'] = data_frame['file'].astype('category')
    data_frame['sentence'] = data_frame['sentence'].astype('category')
    data_frame['speaker'] = data_frame['speaker'].astype('category')

In [4]:
whatsapp_data_train = load_files_with_prev_next("./../data/whatsapp_train_data/")
whatsapp_data_test = load_files_with_prev_next("./../data/whatsapp_test_data/")
convert_types(whatsapp_data_train)
convert_types(whatsapp_data_test)
whatsapp_data_test

,file,speaker,sentence,previous_packet,next_packet,packet_size,phonemes,words
0,DR1-FAKS0-SA1.CSV,DR1-FAKS0,She had your dark suit in greasy wash water al...,0,342,249,"(h#,)","(,)"
1,DR1-FAKS0-SA1.CSV,DR1-FAKS0,She had your dark suit in greasy wash water al...,249,335,342,"(h#,)","(,)"
2,DR1-FAKS0-SA1.CSV,DR1-FAKS0,She had your dark suit in greasy wash water al...,342,303,335,"(h#,)","(,)"
3,DR1-FAKS0-SA1.CSV,DR1-FAKS0,She had your dark suit in greasy wash water al...,335,364,303,"(h#, sh)","(she,)"
4,DR1-FAKS0-SA1.CSV,DR1-FAKS0,She had your dark suit in greasy wash water al...,303,418,364,"(sh, iy, hv)","(she, had)"
...,...,...,...,...,...,...,...,...
31584,DR8-MSLB0-SX383.CSV,DR8-MSLB0,The carpet cleaners shampooed our oriental rug.,338,370,303,"(r, ao, r)","(our, oriental)"
31585,DR8-MSLB0-SX383.CSV,DR8-MSLB0,The carpet cleaners shampooed our oriental rug.,303,314,370,"(r, iy, eh)","(oriental,)"
31586,DR8-MSLB0-SX383.CSV,DR8-MSLB0,The carpet cleaners shampooed our oriental rug.,370,303,314,"(eh, n, tcl, t)","(oriental,)"
31587,DR8-MSLB0-SX383.CSV,DR8-MSLB0,The carpet cleaners shampooed our oriental rug.,314,295,303,"(t, el, r, ah)","(oriental, rug)"


In [5]:
def add_surrounding(data_frame):
    data_frame['prev_curr'] = list(zip(data_frame.previous_packet, data_frame.packet_size))
    data_frame['next_curr'] = list(zip(data_frame.next_packet, data_frame.packet_size))
    data_frame['packet_surrounding'] = list(zip(data_frame.previous_packet, data_frame.packet_size, data_frame.next_packet))
    
    #data_frame['prev_curr'] = data_frame['prev_curr'].astype('category')
    #data_frame['next_curr'] = data_frame['next_curr'].astype('category')
    #data_frame['packet_surrounding'] = data_frame['packet_surrounding'].astype('category')

add_surrounding(whatsapp_data_train)
add_surrounding(whatsapp_data_test)

whatsapp_data_train = whatsapp_data_train[['file', 'speaker', 'sentence', 'previous_packet', 'next_packet','packet_size', 'prev_curr', 'next_curr', 'packet_surrounding', 'phonemes', 'words']]
whatsapp_data_test = whatsapp_data_test[['file', 'speaker', 'sentence', 'previous_packet', 'next_packet','packet_size', 'prev_curr', 'next_curr', 'packet_surrounding', 'phonemes', 'words']]
whatsapp_data_train

,file,speaker,sentence,previous_packet,next_packet,packet_size,prev_curr,next_curr,packet_surrounding,phonemes,words
0,DR1-FCJF0-SA1.CSV,DR1-FCJF0,She had your dark suit in greasy wash water al...,0,380,227,"(0, 227)","(380, 227)","(0, 227, 380)","(h#,)","(,)"
1,DR1-FCJF0-SA1.CSV,DR1-FCJF0,She had your dark suit in greasy wash water al...,227,407,380,"(227, 380)","(407, 380)","(227, 380, 407)","(h#, sh, ix)","(she,)"
2,DR1-FCJF0-SA1.CSV,DR1-FCJF0,She had your dark suit in greasy wash water al...,380,350,407,"(380, 407)","(350, 407)","(380, 407, 350)","(ix, hv, eh)","(she, had)"
3,DR1-FCJF0-SA1.CSV,DR1-FCJF0,She had your dark suit in greasy wash water al...,407,281,350,"(407, 350)","(281, 350)","(407, 350, 281)","(eh, dcl, jh)","(had, your)"
4,DR1-FCJF0-SA1.CSV,DR1-FCJF0,She had your dark suit in greasy wash water al...,350,327,281,"(350, 281)","(327, 281)","(350, 281, 327)","(jh, ih, dcl, d, ah)","(had, your, dark)"
...,...,...,...,...,...,...,...,...,...,...,...
86492,DR8-MTCS0-SX82.CSV,DR8-MTCS0,Good service should be rewarded by big tips.,286,253,268,"(286, 268)","(253, 268)","(286, 268, 253)","(ay, bcl, b, ih)","(by, big)"
86493,DR8-MTCS0-SX82.CSV,DR8-MTCS0,Good service should be rewarded by big tips.,268,315,253,"(268, 253)","(315, 253)","(268, 253, 315)","(ih, gcl)","(big,)"
86494,DR8-MTCS0-SX82.CSV,DR8-MTCS0,Good service should be rewarded by big tips.,253,279,315,"(253, 315)","(279, 315)","(253, 315, 279)","(gcl, t, ih)","(big, tips)"
86495,DR8-MTCS0-SX82.CSV,DR8-MTCS0,Good service should be rewarded by big tips.,315,392,279,"(315, 279)","(392, 279)","(315, 279, 392)","(ih, pcl, p)","(tips,)"


### Preparing data

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [7]:
# add removal of labels for the test_dataset
def get_labels(df, label=["words"], feature=["previous_packet", "packet_size", "next_packet"]):
    labels = df[label]
    features = df[feature]
    return features, labels

In [8]:
def prepare_labels(train_labels, test_labels, label=["words"]):
    train_labels = train_labels.astype('category')
    test_labels = test_labels.astype('category')
    
    total_labels = train_labels.append(test_labels)
    
    lab_enc = LabelEncoder()
    lab_enc.fit(total_labels[label])

    train_labels = lab_enc.transform(train_labels[label])
    test_labels = lab_enc.transform(test_labels[label])
    
    return train_labels, test_labels, lab_enc

In [9]:
train_set, train_labels = get_labels(whatsapp_data_train)
test_set, test_labels = get_labels(whatsapp_data_test)

train_labels = train_labels.astype('category')
test_labels = test_labels.astype('category')

total_labels = train_labels.append(test_labels)
print(len(pd.unique(train_labels.words)))
print(len(pd.unique(test_labels.words)))
total_unique_words = len(pd.unique(total_labels.words))
total_unique_words

16168
6739


21317

Now we can see that we have a really big problem => there are 5149 new words that we have never seen. As we saw in our analysis we can't really generalise on never seen words before.

In [11]:
train_set, train_labels = get_labels(whatsapp_data_train, label=['phonemes'])
test_set, test_labels = get_labels(whatsapp_data_test, label=['phonemes'])

train_labels = train_labels.astype('category')
test_labels = test_labels.astype('category')

total_labels = train_labels.append(test_labels)
print(len(pd.unique(train_labels.phonemes)))
print(len(pd.unique(test_labels.phonemes)))
total_unique_phonemes = len(pd.unique(total_labels.phonemes))
total_unique_phonemes

27369
12655


33990

With phonemes the situation is a bit different, as there are more phonemes and we haven't seen only half of them.

### Tree classifier

First model that we will be trying is tree classifier. In the analysis we have noticed, that there is almost a 1:1 correspondence of trigram of phoneme sizes and words (eg. that for every trigram of phoneme sizes there is different word). 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
train_set, train_labels = get_labels(whatsapp_data_train)
test_set, test_labels = get_labels(whatsapp_data_test)

train_labels = train_labels.astype('category')
test_labels = test_labels.astype('category')

In [9]:
total_labels = train_labels.append(test_labels)
print(len(pd.unique(train_labels.words)))
print(len(pd.unique(test_labels.words)))
len(pd.unique(total_labels.words))

16168
6739


21317

In [10]:
lab_enc = LabelEncoder()
lab_enc.fit(total_labels.words)

train_labels = lab_enc.transform(train_labels.words)
test_labels = lab_enc.transform(test_labels.words)
train_labels

array([    0, 15548, 15578, ...,  2531, 18676, 18676])

In [11]:
tree_clf_pipeline = Pipeline(
    [
        (
            "clf",
            DecisionTreeClassifier(criterion="entropy", max_depth=None, splitter="best",
                                   min_samples_split=2, random_state=42),
        ),
    ]
)
# Words: criterion="entropy", max_depth=None, splitter="best", min_samples_split=2, random_state=42 => 0.97, 0.02
# 

In [12]:
print("Starting!")
tree_clf_pipeline.fit(train_set, train_labels)
print("Finished!")

print(f"Train accuracy: {tree_clf_pipeline.score(train_set, train_labels):.4f}")
print(f"Test accuracy : {tree_clf_pipeline.score(test_set, test_labels):.4f}")

Train accuracy: 0.2631
Test accuracy : 0.0380


Phonemes

In [15]:
train_set, train_labels = get_labels(whatsapp_data_train, label=["phonemes"])
test_set, test_labels = get_labels(whatsapp_data_test, label=["phonemes"])

train_labels, test_labels, _ = prepare_labels(train_labels, test_labels, label=["phonemes"])

array([12858, 13445, 15553, ..., 12837, 14816, 24059])

In [16]:
print("Starting!")
tree_clf_pipeline.fit(train_set, train_labels)
print("Finished!")

print(f"Train accuracy: {tree_clf_pipeline.score(train_set, train_labels):.4f}")
print(f"Test accuracy : {tree_clf_pipeline.score(test_set, test_labels):.4f}")

Train accuracy: 0.2454
Test accuracy : 0.0293


I have been able to run these classificators and the best results I was able to get were around 3%, which isn't that good considering KNN was able to get twice that much.

### KNN

Let's take a look a different kind of classifier => k nearest neighbours. This classifier shouldn't need that much RAM and that much of a computational power.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [10]:
train_set, train_labels = get_labels(whatsapp_data_train, label=["words"])
test_set, test_labels = get_labels(whatsapp_data_test, label=["words"])

train_labels, test_labels, _ = prepare_labels(train_labels, test_labels)

c:\swdevelopment\python38\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [17]:
knn_clf_pipeline = Pipeline(
    [
        (
            "clf",
            KNeighborsClassifier(20, weights='distance', n_jobs=4)
        ),
    ]
)

# train accuracy 0.9723
#5   => 0.0313
#10  => 0.0377
#20  => 0.0450
#32  => 0.0497
#64  => 0.0567
#128 => 0.0625
#256 => 0.0668

# uniform gives better test results but doesn't seem to be able to "answer correctly" on the train test
# 64 => 0.0927, 0.0685

In [18]:
print("Starting!")
knn_clf_pipeline.fit(train_set, train_labels)
print("Finished!")

print(f"Train accuracy: {knn_clf_pipeline.score(train_set, train_labels):.4f}")
print(f"Test accuracy : {knn_clf_pipeline.score(test_set, test_labels):.4f}")

Train accuracy: 0.9723
Test accuracy : 0.0438


For the search space of 64 nearest neighbours we get only 5.67% success rate on our test data (which is around 1971 words). I have listed other parameters and their resulting percentages in the comments in the code cell. Also worth noting is that "StandardScaler" only worsens our predictions.

This whole section is just made as sanity check that we actually get expected results (that is we only guess the words we've already seen and none from which we haven't seen).

Probably remove -----

In [20]:
data_test_copy = whatsapp_data_test.copy()

column_select = list(map(lambda x: x in list(whatsapp_data_train.words.drop_duplicates()), list(data_test_copy.words)))

print("Known words:\t", column_select.count(True))
print("Unknown words:\t", column_select.count(False))

Known words:	 16405
Unknown words:	 15184


In [21]:
data_test_copy = data_test_copy[column_select]

In [22]:
train_set, train_labels = get_labels(whatsapp_data_train, label=["words"])
test_set, test_labels = get_labels(data_test_copy, label=["words"])

train_labels, test_labels, _ = prepare_labels(train_labels, test_labels)

c:\swdevelopment\python38\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [23]:
print("Starting!")
# knn_clf_pipeline.fit(train_set, train_labels)
print("Finished!")

# 256, distance => 0.9723, 0.1286 on only 

print(f"Train accuracy: {knn_clf_pipeline.score(train_set, train_labels):.4f}")
print(f"Test accuracy : {knn_clf_pipeline.score(test_set, test_labels):.4f}")

Train accuracy: 0.0516
Test accuracy : 0.0518


From this we get that the succes rate on known is around double the ammount on all words (this can be seen from the output of a cell 2 cells above).

In [24]:
train_set, train_labels = get_labels(whatsapp_data_train, label=["words"])
test_set, test_labels = get_labels(whatsapp_data_test[list(map(lambda x: not x, column_select))], label=["words"])

train_labels, test_labels, _ = prepare_labels(train_labels, test_labels)

c:\swdevelopment\python38\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [25]:
print("Starting!")
#knn_clf_pipeline.fit(train_set, train_labels)
print("Finished!")

print(f"Train accuracy: {knn_clf_pipeline.score(train_set, train_labels):.4f}")
print(f"Test accuracy : {knn_clf_pipeline.score(test_set, test_labels):.4f}")

Train accuracy: 0.9723
Test accuracy : 0.0000


This test was only made as "sanity check" as it is indeed highly probable that our model wouldn't be able to properly guess on never seen examples of words.

To here remove -------

Now let's try our luck with phonemes:

In [26]:
train_set, train_labels = get_labels(whatsapp_data_train, label=["phonemes"])
test_set, test_labels = get_labels(whatsapp_data_test, label=["phonemes"])

train_labels, test_labels, _ = prepare_labels(train_labels, test_labels, label=["phonemes"])

c:\swdevelopment\python38\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [27]:
knn_clf_pipeline = Pipeline(
    [
        (
            "clf",
            KNeighborsClassifier(16, weights='distance', n_jobs=4)
        ),
    ]
)
# 5 => 0.0256
# 6 => 0.0269
# 10 => 0.0299
# 20 => 0.0343
# 32 => 0.0368
# 64 => 0.0404
# 128 => 0.0441
# 256 => 0.0465

In [28]:
print("Starting!")
knn_clf_pipeline.fit(train_set, train_labels)
print("Finished!")

print(f"Train accuracy: {knn_clf_pipeline.score(train_set, train_labels):.4f}")
print(f"Test accuracy : {knn_clf_pipeline.score(test_set, test_labels):.4f}")

Train accuracy: 0.9723
Test accuracy : 0.0324


We can clearly see that phonemes didn't help us that much and that the results are far worse from those gotten by exploring words.

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
train_set, train_labels = get_labels(whatsapp_data_train, label=["words"])
test_set, test_labels = get_labels(whatsapp_data_test, label=["words"])

train_labels, test_labels, _ = prepare_labels(train_labels, test_labels)

/home/xkadlec6/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [10]:
rfc_pipeline = Pipeline(
    [
        (
            "clf",
            RandomForestClassifier(max_depth=12, random_state=42, criterion = 'entropy', n_jobs = -1, min_samples_split = 2)
        ),
    ]
)

# (max_depth=12, random_state=42, criterion = 'entropy', n_jobs = -1, min_samples_split = 2) => 0.6181, 0.0651

In [11]:
print("Starting!")
rfc_pipeline.fit(train_set, train_labels)
print("Finished!")

print(f"Train accuracy: {rfc_pipeline.score(train_set, train_labels):.4f}")
print(f"Test accuracy : {rfc_pipeline.score(test_set, test_labels):.4f}")

Starting!
Finished!
Train accuracy: 0.6181
Test accuracy : 0.0651


We can see, that this indeed has better accuracy than normal tree / KNN, but takes way more system resources.

### AdaBoost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [9]:
train_set, train_labels = get_labels(whatsapp_data_train, label=["words"])
test_set, test_labels = get_labels(whatsapp_data_test, label=["words"])

train_labels, test_labels, _ = prepare_labels(train_labels, test_labels)

/home/xkadlec6/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [10]:
abc_pipeline = Pipeline(
    [
        (
            "clf",
            AdaBoostClassifier(random_state=1, n_estimators = 60, learning_rate=0.9)
        ),
    ]
)


In [1]:
print("Starting!")
#abc_pipeline.fit(train_set, train_labels)
print("Finished!")

#print(f"Train accuracy: {abc_pipeline.score(train_set, train_labels):.4f}")
#print(f"Test accuracy : {abc_pipeline.score(test_set, test_labels):.4f}")

print("0.0505, 0.0471")

0.0505, 0.0471


This classifier ended in absolute failure as it wasn't able to get even acceptable results on the train data. And it even took 8 hours to learn (this is because it can only use 1 thread), so this classifier is pretty much worthless to us.

### MLP Classifier

In [12]:
import keras
import tensorflow as tf

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report

In [9]:
train_set, train_labels = get_labels(whatsapp_data_train, label=["words"])
test_set, test_labels = get_labels(whatsapp_data_test, label=["words"])

train_labels, test_labels, _ = prepare_labels(train_labels, test_labels)

/home/xkadlec6/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [10]:
print(train_set.shape)
print(train_labels.shape)

(86497, 3)
(86497,)


In [11]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels, num_classes=total_unique_words)
test_labels = to_categorical(test_labels, num_classes=total_unique_words)
print(train_labels.shape)

(86497, 21317)


In [12]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(units=512, activation='relu', input_dim=3*1))  # first hidden layer
model.add(Dense(units=256, activation='relu'))  # second hidden layer
# model.add(Dense(units=128, activation='relu'))  # third hidden layer
model.add(Dense(units=total_unique_words, activation='softmax'))  # output layer
# model.add(Dense(units=128))  # output layer

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 21317)             5478469   
Total params: 5,611,845
Trainable params: 5,611,845
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(train_set, train_labels, epochs=64, batch_size=256)

Epoch 1/1024
338/338 [==============================] - 11s 32ms/step - loss: 8.7646 - accuracy: 0.0468
Epoch 2/1024
338/338 [==============================] - 12s 35ms/step - loss: 7.8913 - accuracy: 0.0528
Epoch 3/1024
338/338 [==============================] - 10s 30ms/step - loss: 7.7098 - accuracy: 0.0534
Epoch 4/1024
338/338 [==============================] - 10s 30ms/step - loss: 7.6375 - accuracy: 0.0544
Epoch 5/1024
338/338 [==============================] - 10s 30ms/step - loss: 7.5539 - accuracy: 0.0564
Epoch 6/1024
338/338 [==============================] - 10s 29ms/step - loss: 7.5238 - accuracy: 0.0560
Epoch 7/1024
338/338 [==============================] - 10s 29ms/step - loss: 7.4832 - accuracy: 0.0558
Epoch 8/1024
338/338 [==============================] - 10s 30ms/step - loss: 7.4430 - accuracy: 0.0568
Epoch 9/1024
338/338 [==============================] - 10s 30ms/step - loss: 7.4139 - accuracy: 0.0598
Epoch 10/1024
338/338 [==============================] - 11s 32m

338/338 [==============================] - 9s 28ms/step - loss: 6.6561 - accuracy: 0.0730
Epoch 157/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.6548 - accuracy: 0.0732
Epoch 158/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.6537 - accuracy: 0.0738
Epoch 159/1024
338/338 [==============================] - 10s 28ms/step - loss: 6.6444 - accuracy: 0.0742
Epoch 160/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.6471 - accuracy: 0.0726
Epoch 161/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.6373 - accuracy: 0.0751
Epoch 162/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.6485 - accuracy: 0.0743
Epoch 163/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.6305 - accuracy: 0.0735
Epoch 164/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.6474 - accuracy: 0.0737
Epoch 165/1024
338/338 [==============================] - 9s 27ms/ste

338/338 [==============================] - 10s 29ms/step - loss: 6.4841 - accuracy: 0.0758
Epoch 310/1024
338/338 [==============================] - 9s 26ms/step - loss: 6.4708 - accuracy: 0.0755
Epoch 311/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.4723 - accuracy: 0.0753
Epoch 312/1024
338/338 [==============================] - 10s 28ms/step - loss: 6.4811 - accuracy: 0.0737
Epoch 313/1024
338/338 [==============================] - 9s 26ms/step - loss: 6.4666 - accuracy: 0.0756
Epoch 314/1024
338/338 [==============================] - 9s 25ms/step - loss: 6.4598 - accuracy: 0.0776
Epoch 315/1024
338/338 [==============================] - 8s 24ms/step - loss: 6.4921 - accuracy: 0.0734  - ETA
Epoch 316/1024
338/338 [==============================] - 9s 26ms/step - loss: 6.4758 - accuracy: 0.0744
Epoch 317/1024
338/338 [==============================] - 8s 24ms/step - loss: 6.4668 - accuracy: 0.0749
Epoch 318/1024
338/338 [==============================] - 9s 

338/338 [==============================] - 9s 26ms/step - loss: 6.4276 - accuracy: 0.0741
Epoch 465/1024
338/338 [==============================] - 9s 26ms/step - loss: 6.3996 - accuracy: 0.0763
Epoch 466/1024
338/338 [==============================] - 9s 25ms/step - loss: 6.4184 - accuracy: 0.0745
Epoch 467/1024
338/338 [==============================] - 9s 28ms/step - loss: 6.3960 - accuracy: 0.0755
Epoch 468/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.4042 - accuracy: 0.0743
Epoch 469/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.4062 - accuracy: 0.0735
Epoch 470/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.4027 - accuracy: 0.0753
Epoch 471/1024
338/338 [==============================] - 9s 28ms/step - loss: 6.4343 - accuracy: 0.0728
Epoch 472/1024
338/338 [==============================] - 9s 26ms/step - loss: 6.4059 - accuracy: 0.0752
Epoch 473/1024
338/338 [==============================] - 9s 28ms/step

338/338 [==============================] - 9s 27ms/step - loss: 6.3727 - accuracy: 0.0760
Epoch 619/1024
338/338 [==============================] - 9s 26ms/step - loss: 6.3869 - accuracy: 0.0743
Epoch 620/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.3801 - accuracy: 0.0767
Epoch 621/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.3760 - accuracy: 0.0758
Epoch 622/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.3771 - accuracy: 0.0763
Epoch 623/1024
338/338 [==============================] - 9s 28ms/step - loss: 6.3813 - accuracy: 0.0756
Epoch 624/1024
338/338 [==============================] - 9s 26ms/step - loss: 6.3740 - accuracy: 0.0768
Epoch 625/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.3968 - accuracy: 0.0748
Epoch 626/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.3731 - accuracy: 0.0770
Epoch 627/1024
338/338 [==============================] - 9s 27ms/step

338/338 [==============================] - 9s 26ms/step - loss: 6.3668 - accuracy: 0.0749
Epoch 774/1024
338/338 [==============================] - 9s 28ms/step - loss: 6.3598 - accuracy: 0.0755
Epoch 775/1024
338/338 [==============================] - 9s 28ms/step - loss: 6.3581 - accuracy: 0.0765
Epoch 776/1024
338/338 [==============================] - 9s 28ms/step - loss: 6.3586 - accuracy: 0.0769
Epoch 777/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.3693 - accuracy: 0.0753
Epoch 778/1024
338/338 [==============================] - 8s 25ms/step - loss: 6.3600 - accuracy: 0.0748
Epoch 779/1024
338/338 [==============================] - 9s 26ms/step - loss: 6.3533 - accuracy: 0.0760
Epoch 780/1024
338/338 [==============================] - 9s 27ms/step - loss: 6.3605 - accuracy: 0.0754
Epoch 781/1024
338/338 [==============================] - 9s 26ms/step - loss: 6.3658 - accuracy: 0.0735
Epoch 782/1024
338/338 [==============================] - 9s 26ms/step

338/338 [==============================] - 11s 32ms/step - loss: 6.3467 - accuracy: 0.0767
Epoch 928/1024
338/338 [==============================] - 11s 31ms/step - loss: 6.3525 - accuracy: 0.0745
Epoch 929/1024
338/338 [==============================] - 10s 28ms/step - loss: 6.3403 - accuracy: 0.0782
Epoch 930/1024
338/338 [==============================] - 9s 28ms/step - loss: 6.3557 - accuracy: 0.0758
Epoch 931/1024
338/338 [==============================] - 9s 25ms/step - loss: 6.3525 - accuracy: 0.0744
Epoch 932/1024
338/338 [==============================] - 10s 30ms/step - loss: 6.3511 - accuracy: 0.0747
Epoch 933/1024
338/338 [==============================] - 11s 33ms/step - loss: 6.3326 - accuracy: 0.0765
Epoch 934/1024
338/338 [==============================] - 11s 33ms/step - loss: 6.3459 - accuracy: 0.0765
Epoch 935/1024
338/338 [==============================] - 11s 33ms/step - loss: 6.3508 - accuracy: 0.0782
Epoch 936/1024
338/338 [==============================] - 11s 3

338/338 [==============================] - 11s 31ms/step - loss: 6.3443 - accuracy: 0.0755
Epoch 1005/1024
338/338 [==============================] - 10s 31ms/step - loss: 6.3387 - accuracy: 0.0753
Epoch 1006/1024
338/338 [==============================] - 11s 31ms/step - loss: 6.3349 - accuracy: 0.0760
Epoch 1007/1024
338/338 [==============================] - 10s 30ms/step - loss: 6.3431 - accuracy: 0.0770
Epoch 1008/1024
338/338 [==============================] - 11s 32ms/step - loss: 6.3407 - accuracy: 0.0758
Epoch 1009/1024
338/338 [==============================] - 10s 30ms/step - loss: 6.3402 - accuracy: 0.0749
Epoch 1010/1024
338/338 [==============================] - 11s 32ms/step - loss: 6.3368 - accuracy: 0.0753
Epoch 1011/1024
338/338 [==============================] - 10s 31ms/step - loss: 6.3334 - accuracy: 0.0761
Epoch 1012/1024
338/338 [==============================] - 11s 32ms/step - loss: 6.3429 - accuracy: 0.0758
Epoch 1013/1024
338/338 [============================

In [15]:
print("test loss, test acc:", model.evaluate(test_set, test_labels))

988/988 [==============================] - 4s 4ms/step - loss: 28.1301 - accuracy: 0.0721
test loss, test acc: [28.130123138427734, 0.07205040007829666]


batch size = 128:  
test loss, test acc: [15.796355247497559, 0.0679350420832634] => 50 epochs  
test loss, test acc: [18.180967330932617, 0.06521257758140564] => 100 epochs

batch size = 256:  
test loss, test acc: [21.200485229492188, 0.071987085044384] => 256 epochs
test loss, test acc: [28.130123138427734, 0.07205040007829666] => 1024 epochs

In [16]:
"""
pred_y = model.predict(test_set)
print(len(pred_y))
print(pred_y[0])

pred_y_labels = [0]*len(pred_y)
for i in range(len(pred_y)):
    pred_y_labels[i] = np.argmax(pred_y[i])
    
print(pred_y_labels[0])

print(classification_report(test_labels, pred_y_labels))
"""
print("Not used")

#### Phonemes

In [14]:
train_set, train_labels = get_labels(whatsapp_data_train, label=["phonemes"])
test_set, test_labels = get_labels(whatsapp_data_test, label=["phonemes"])

train_labels, test_labels, _ = prepare_labels(train_labels, test_labels, label=["phonemes"])

/home/xkadlec6/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [15]:
print(train_set.shape)
print(train_labels.shape)

(86497, 3)
(86497,)


In [16]:
train_labels = to_categorical(train_labels, num_classes=total_unique_phonemes)
test_labels = to_categorical(test_labels, num_classes=total_unique_phonemes)
print(train_labels.shape)

(86497, 33990)


In [20]:
model = Sequential()

model.add(Dense(units=512, activation='relu', input_dim=3*1))  # first hidden layer
model.add(Dense(units=256, activation='relu'))  # second hidden layer
# model.add(Dense(units=128, activation='relu'))  # third hidden layer
model.add(Dense(units=total_unique_phonemes, activation='softmax'))  # output layer
# model.add(Dense(units=128))  # output layer

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               2048      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 33990)             8735430   
Total params: 8,868,806
Trainable params: 8,868,806
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(train_set, train_labels, epochs=128, batch_size=256)

Epoch 1/128
338/338 [==============================] - 37s 108ms/step - loss: 9.4086 - accuracy: 0.0408
Epoch 2/128
338/338 [==============================] - 37s 108ms/step - loss: 8.5344 - accuracy: 0.0470
Epoch 3/128
338/338 [==============================] - 36s 107ms/step - loss: 8.2638 - accuracy: 0.0471
Epoch 4/128
338/338 [==============================] - 36s 106ms/step - loss: 8.1087 - accuracy: 0.0468
Epoch 5/128
338/338 [==============================] - 35s 105ms/step - loss: 8.0128 - accuracy: 0.0472
Epoch 6/128
338/338 [==============================] - 36s 106ms/step - loss: 7.9068 - accuracy: 0.0489
Epoch 7/128
338/338 [==============================] - 35s 105ms/step - loss: 7.8324 - accuracy: 0.0475
Epoch 8/128
338/338 [==============================] - 35s 103ms/step - loss: 7.7745 - accuracy: 0.0475
Epoch 9/128
338/338 [==============================] - 36s 105ms/step - loss: 7.7123 - accuracy: 0.0473
Epoch 10/128
338/338 [==============================] - 35s 104m

In [22]:
print("test loss, test acc:", model.evaluate(test_set, test_labels))

988/988 [==============================] - 7s 5ms/step - loss: 20.6821 - accuracy: 0.0495
test loss, test acc: [20.68207550048828, 0.049479249864816666]


batch_size 256:  
test loss, test acc: [9.716291427612305, 0.04188166931271553] => 4 epochs  
test loss, test acc: [20.68207550048828, 0.049479249864816666] => 128 epochs

### LSTM